This shall be the notebook for constructing different kinds of datasets concerning the preprocessing of the data. 


In [1]:
import numpy as np
!pip install -q pandas
import pandas as pd

import matplotlib as plt
# from google.colab import drive
import random
random.seed(138)
!pip install -q sklearn

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 23.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Command "python setup.py egg_info" failed with error code 1 in C:\Users\Jurekk\AppData\Local\Temp\pip-install-syq2s7ec\sklearn\
You are using pip version 10.0.1, however version 23.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Load in data. 

In [36]:
# define functions that are going to be helpful for the preprocessing 

def Angle(long, lat):
    if long > 0 and lat > 0:
        long = abs(long)
        lat = abs(lat)
        return np.arctan(long / lat)* 180/np.pi + 180

    if long < 0 and lat < 0:
        long = abs(long)
        lat = abs(lat)
        return np.arctan(long / lat)* 180/np.pi
    
    if long < 0 and lat > 0:
        long = abs(long)
        lat = abs(lat)
        return 180 - np.arctan(long / lat)* 180/np.pi
    
    if long > 0 and lat < 0:
        long = abs(long)
        lat = abs(lat)
        return 360 - np.arctan(long / lat)* 180/np.pi
    

def wind_direction(long, lat):
    vectorAngle = np.vectorize(Angle)
    wind_direction = vectorAngle(long, lat)
    return wind_direction


# in order to have the target precipitation data also in timesteps of 6hrs, sum up the first 6 values of precip (each corresponding to the total precip )
def accumulate_precip(precip_data):
    accumulated = pd.Series(precip_data.rolling(6).sum())
    return accumulated



In [61]:
df_precip = pd.read_csv('ECMWF_2017_2018_precip.csv')
df_surface = pd.read_csv('ECMWF_2017_2018_surface.csv')

df_2017_true = pd.read_csv('synop_2017_March_June.csv')
df_2018_true = pd.read_csv('synop_2018_March_June.csv')

# only use the stuff we need, rename valid_time to date time 

df_precip = df_precip[['number', 'valid_time', 'tp6']]
df_surface = df_surface[['number', 'valid_time', 'u10', 'v10', 't2m']]
df_surface['wind_speed'] = np.sqrt(df_surface['u10']**2 + df_surface['v10']**2)
# df_surface['wind_direction'] = np.zeros(len(df_surface))

# calculate the wind direction
df_surface['wind_direction'] = wind_direction(df_surface['u10'], df_surface['v10'])

# fallunterscheidung for different cases for signs of u10 and v10

# for i in range(int(len(df_surface)/100)):
#     if i%10000 == 0:
#       print(i)
#     u10 = df_surface.u10[i]
#     v10 = df_surface.v10[i]
#     if u10 > 0 and v10 > 0:
#         df_surface['windangle'][i] = 180 + np.arctan(abs(u10/v10)) * 180/np.pi
#     elif u10 < 0 and v10 > 0:
#         df_surface['windangle'][i] = 180 - np.arctan(abs(u10/v10)) * 180/np.pi
#     elif u10 < 0 and v10 < 0:
#         df_surface['windangle'][i] = np.arctan(abs(u10/v10)) * 180/np.pi
#     elif u10 > 0 and v10 < 0:
#         df_surface['windangle'][i] = 360 - np.arctan(abs(u10/v10)) * 180/np.pi

# df_surface = df_surface.iloc[::21, :].pop
# in precip, the first 00:00:00 is missing for every day. 
df_surface = df_surface[df_surface.index % 21 != 0]
df_surface = df_surface.reset_index(drop=True)
# these two now are equally long

df_precip = df_precip.rename(columns={'valid_time':'datetime'})
df_surface = df_surface.rename(columns={'valid_time':'datetime'})

# df_precip.to_csv('forecastdata_precip.csv', index = False)
# df_surface.to_csv('forecastdata_surface.csv', index = False)

# create one dataframe containing the input

df_input = df_surface[['datetime', 'number', 'wind_speed', 'wind_direction', 't2m']]
# we want this in mm
df_input['tp6'] =  df_precip[['tp6']]*1000
df_input['t2m'] = df_input['t2m']-273.15
df_input.to_csv('forecastdata.csv', index = False)
# print(str(len(df_precip)) + ' and ' + str(len(df_surface)))



df_2017_true = df_2017_true[['datetime', 'temp', 'wind_direction', 'wind_speed', 'precip_quantity_1hour', 'local_datetime']]
df_2018_true = df_2018_true[['datetime', 'temp', 'wind_direction', 'wind_speed', 'precip_quantity_1hour', 'local_datetime']]

df_true = pd.concat([df_2017_true, df_2018_true], ignore_index=True, axis=0)
# keep only every second value
df_true = df_true.iloc[::2, :]
# add the values of the precip up
df_true['precip_quantity_6hour'] = np.append(np.array([0]), accumulate_precip(df_true['precip_quantity_1hour']))[:-1]
df_true = df_true.iloc[::6, :]
df_true = df_true.drop(columns=['precip_quantity_1hour'])
# drop first row
df_true = df_true.tail(-1)
df_true.to_csv('true_data.csv', index = False)
# print(len(df_2017_true), len(df_2018_true), len(df_true))


c:\Users\Jurekk\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\Users\Jurekk\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,t2m,wind_direction,wind_speed,tp6
0,5.29824,256.721039,6.551333,1.165867
1,8.48452,241.731494,7.895033,0.235081
2,6.03726,174.258163,4.422257,3.531933
3,7.29238,245.838730,8.591761,2.343178
4,5.59506,253.335915,9.110837,0.010014
5,8.97720,270.564778,10.006492,0.130653
6,6.51968,244.374691,4.953512,0.139236
7,1.65652,183.708835,2.833401,0.000000
8,3.89680,151.557149,4.853381,0.000000
9,8.67290,158.076926,4.890584,0.012398


We have now two dataframes, one containing all the inputs, where we deleted all the 00:00:00 values for the first day of the forecast. 
We have 20 values for every day. For 120 days. And this 50 times. For 2 years. So the length of our dataframe is 20 x 50 x 120 x 2, which makes sense.

The length of the true dataframe is 120 days x 2 years x 4 measurements per day = 960 which also kinda makes sense

There's two strategies concerning the processing of the data that we are going to follow: 

Firstly, create a data set with 12 columns where we have 3 values for each variable. T-1, T and T+1. For the first one in a pack of 20 we use T, T, T+1 and for the last ones T-1, T, T respectively. 

Secondly, the approach shall be constructing a dataset with 50 inputs for each variable. So either three nets with each 50 inputnodes (or even 200, but we only predict one) or one net with 200 input nodes and three output nodes. 

First the first. 


In [73]:
# let's construct this bad boy 
df_input = pd.read_csv('forecastdata.csv')
df_true = pd.read_csv('true_data.csv')
days = 244
timesteps = 20
no_ensembles = 1

arr = np.zeros((len(df_input), 12))


# for the target data
arr_target = np.zeros((len(df_input), 4))


for i in range(no_ensembles):
    print(i)
    for j in range(days):
        for k in range(timesteps): 
            if k > 0 and k < timesteps - 1:
                arr[i*days*timesteps + j*timesteps + k, 0:4] = df_input[['t2m', 'wind_direction', 'wind_speed', 'tp6']].iloc[i*days*timesteps + j*timesteps + k-1]
                arr[i*days*timesteps + j*timesteps + k, 4:8] = df_input[['t2m', 'wind_direction', 'wind_speed', 'tp6']].iloc[i*days*timesteps + j*timesteps + k]
                arr[i*days*timesteps + j*timesteps + k, 8:12] = df_input[['t2m', 'wind_direction', 'wind_speed', 'tp6']].iloc[i*days*timesteps + j*timesteps + k+1]

                # it just runs through true array, taking 1, 2, 3, ..., 19, 20, 2, 3, 4, ... , 20, 21, 3, 4, 5, ...  and so on 
                # once we hit the next ensemble after 2years, so 244 days, we jump back to the first element again
                arr_target[i*days*timesteps + j*timesteps + k, :] = df_true[['temp', 'wind_direction', 'wind_speed', 'precip_quantity_6hour']].iloc[j + k]

            elif k == 0:
                arr[i*days*timesteps + j*timesteps + k, 0:4] = df_input[['t2m', 'wind_direction', 'wind_speed', 'tp6']].iloc[i*days*timesteps + j*timesteps + k]
                arr[i*days*timesteps + j*timesteps + k, 4:8] = df_input[['t2m', 'wind_direction', 'wind_speed', 'tp6']].iloc[i*days*timesteps + j*timesteps + k]
                arr[i*days*timesteps + j*timesteps + k, 8:12] = df_input[['t2m', 'wind_direction', 'wind_speed', 'tp6']].iloc[i*days*timesteps + j*timesteps + k+1]

                # it just runs through true array, taking 1, 2, 3, ..., 19, 20, 2, 3, 4, ... , 20, 21, 3, 4, 5, ...  and so on 
                # once we hit the next ensemble after 2years, so 244 days, we jump back to the first element again
                arr_target[i*days*timesteps + j*timesteps + k, :] = df_true[['temp', 'wind_direction', 'wind_speed', 'precip_quantity_6hour']].iloc[j + k]
               
            elif (k == timesteps-1) :
                arr[i*days*timesteps + j*timesteps + k, 0:4] = df_input[['t2m', 'wind_direction', 'wind_speed', 'tp6']].iloc[i*days*timesteps + j*timesteps + k-1]
                arr[i*days*timesteps + j*timesteps + k, 4:8] = df_input[['t2m', 'wind_direction', 'wind_speed', 'tp6']].iloc[i*days*timesteps + j*timesteps + k]
                arr[i*days*timesteps + j*timesteps + k, 8:12] = df_input[['t2m', 'wind_direction', 'wind_speed', 'tp6']].iloc[i*days*timesteps + j*timesteps + k]

                # it just runs through true array, taking 1, 2, 3, ..., 19, 20, 2, 3, 4, ... , 20, 21, 3, 4, 5, ...  and so on 
                # once we hit the next ensemble after 2years, so 244 days, we jump back to the first element again
                arr_target[i*days*timesteps + j*timesteps + k, :] = df_true[['temp', 'wind_direction', 'wind_speed', 'precip_quantity_6hour']].iloc[j + k]



0


KeyboardInterrupt: 

In [71]:
df_target = pd.DataFrame(arr_target, columns = ['temp', 'wind_direction', 'wind_speed', 'precip_quantity_6hour'])
df_training = pd.DataFrame(arr, columns = ['T-1', 'WD-1', 'WS-1', 'Pr-1', 'T', 'WD', 'WS', 'Pr', 'T+1', 'WD+1', 'WS+1', 'Pr+1'])

df_training[['t2m_true', 'wind_direction_true', 'wind_speed_true', 'precip_quantity_6hour_true']] = df_target[['temp', 'wind_direction', 'wind_speed', 'precip_quantity_6hour']]


df_training.head()

,T-1,WD-1,WS-1,Pr-1,T,WD,WS,Pr,T+1,WD+1,WS+1,Pr+1,t2m_true,wind_direction_true,wind_speed_true,precip_quantity_6hour_true
0,5.29824,256.721039,6.551333,1.165867,5.29824,256.721039,6.551333,1.165867,8.48452,241.731494,7.895033,0.235081,5.7,232.1,3.219,2.0
1,5.29824,256.721039,6.551333,1.165867,8.48452,241.731494,7.895033,0.235081,6.03726,174.258163,4.422257,3.531933,7.7,218.3,5.468,0.4
2,8.48452,241.731494,7.895033,0.235081,6.03726,174.258163,4.422257,3.531933,7.29238,245.838730,8.591761,2.343178,5.9,155.5,3.412,0.4
3,6.03726,174.258163,4.422257,3.531933,7.29238,245.838730,8.591761,2.343178,5.59506,253.335915,9.110837,0.010014,7.5,231.7,4.591,1.4
4,7.29238,245.838730,8.591761,2.343178,5.59506,253.335915,9.110837,0.010014,8.97720,270.564778,10.006492,0.130653,6.2,246.1,6.096,0.8


In [48]:
# construct the first one 

# def split_in_twenty(data):
#     frames = []
#     for i in range(int(len(data)/20)):
#         frames.append(data[i*20:i*20+20:1])
#     return frames

# def createTrainingSet(input, target):
#     oneTrainingInput=[]
#     oneTarget=[]
#     for frame in input:
#         for i in range(len(frame)):
#             if i > 0 and i < len(frame) - 1:
#                 oneTrainingInput.append(list(frame[["t2m", "wind_direction", "wind_speed", "tp6"]].iloc[i-1])
#                 + list(frame[["t2m", "wind_direction", "wind_speed", "tp6"]].iloc[i])
#                 + list(frame[["t2m", "wind_direction", "wind_speed", "tp6"]].iloc[i+1]))
#             elif i == 0:
#                 oneTrainingInput.append(list(frame[["t2m", "wind_direction", "wind_speed", "tp6"]].iloc[i])
#                 + list(frame[["t2m", "wind_direction", "wind_speed", "tp6"]].iloc[i])
#                 + list(frame[["t2m", "wind_direction", "wind_speed", "tp6"]].iloc[i+1]))
#             else:
#                 oneTrainingInput.append(list(frame[["t2m", "wind_direction", "wind_speed", "tp6"]].iloc[i-1])
#                 + list(frame[["t2m", "wind_direction", "wind_speed", "tp6"]].iloc[i])
#                 + list(frame[["t2m", "wind_direction", "wind_speed", "tp6"]].iloc[i]))
#             time = frame.datetime.iloc[i]
#             correctTarget = target[target.datetime == time][["temp", "wind_direction", "wind_speed", "precip_quantity_6hr"]]
#             oneTarget.append(list(correctTarget.iloc[0]))
#     return pd.DataFrame(list(zip(oneTrainingInput, oneTarget)), columns=["Input", "Target"])

# df_input = split_in_twenty(df_input)
# df_training = createTrainingSet(df_input, df_true)
# df_training.to_csv('trainingset.csv')










TypeError: list indices must be integers or slices, not list

In [43]:
# let's construct this bad boy 
days = 244
timesteps = 20
no_ensembles = 50

arr = np.zeros((len(df_input), 12))
df_training = pd.DataFrame(arr, columns = ['T-1', 'T', 'T+1', 'WD-1', 'WD', 'WD+1', 'WS-1', 'WS', 'WS+1', 'Pr-1', 'Pr', 'Pr+1'])
# df_training[['T-2', 'T-1', 'T', 'T+1', 'T+2', 'WD-2', 'WD-1', 'WD', 'WD+1', 'WD+2', 'WS-2', 'WS-1', 'WS', 'WS+1', 'WS+2', 'Pr-2', 'Pr-1', 'Pr', 'Pr+1', 'Pr+2']]


# for the target data
arr_target = np.zeros((len(df_true), 4))
df_target = pd.DataFrame(arr_target, columns = ['temp', 'wind_direction', 'wind_speed', 'precip_quantity_6hour'])


for element in df_input():
     



for number in range(no_ensembles):
    for day in range(days):
        for step in range(timesteps):
                if step == 0:
                    df_training['T-1'][day*timesteps + step]   =   df_input['t2m'][day*timesteps + step]
                    df_training['T'][day*timesteps + step]     =   df_input['t2m'][day*timesteps + step]
                    df_training['T+1'][day*timesteps + step]   =   df_input['t2m'][day*timesteps + step+1]
                    df_training['WD-1'][day*timesteps + step]  =   df_input['wind_direction'][day*timesteps + step]
                    df_training['WD'][day*timesteps + step]    =   df_input['wind_direction'][day*timesteps + step]
                    df_training['WD+1'][day*timesteps + step]  =   df_input['wind_direction'][day*timesteps + step+1]
                    df_training['WS-1'][day*timesteps + step]  =   df_input['wind_speed'][day*timesteps + step]
                    df_training['WS'][day*timesteps + step]    =   df_input['wind_speed'][day*timesteps + step]
                    df_training['WS+1'][day*timesteps + step]  =   df_input['wind_speed'][day*timesteps + step+1]
                    df_training['Pr-1'][day*timesteps + step]  =   df_input['tp6'][day*timesteps + step]
                    df_training['Pr'][day*timesteps + step]    =   df_input['tp6'][day*timesteps + step]
                    df_training['Pr+1'][day*timesteps + step]  =   df_input['tp6'][day*timesteps + step+1]

                    df_target['temp'][day*timesteps + step]                     =   df_true['temp'][day + step] 
                    df_target['wind_direction'][day*timesteps + step]           =   df_true['wind_direction'][day + step] 
                    df_target['wind_speed'][day*timesteps + step]               =   df_true['wind_speed'][day + step] 
                    df_target['precip_quantity_6hour'][day*timesteps + step]    =   df_true['precip_quantity_6hour'][day + step]

                    continue

                if step == timesteps-1:
                    df_training['T-1'][day*timesteps + step]   =   df_input['t2m'][day*timesteps + step-1]
                    df_training['T'][day*timesteps + step]     =   df_input['t2m'][day*timesteps + step]
                    df_training['T+1'][day*timesteps + step]   =   df_input['t2m'][day*timesteps + step]
                    df_training['WD-1'][day*timesteps + step]  =   df_input['wind_direction'][day*timesteps + step-1]
                    df_training['WD'][day*timesteps + step]    =   df_input['wind_direction'][day*timesteps + step]
                    df_training['WD+1'][day*timesteps + step]  =   df_input['wind_direction'][day*timesteps + step]
                    df_training['WS-1'][day*timesteps + step]  =   df_input['wind_speed'][day*timesteps + step-1]
                    df_training['WS'][day*timesteps + step]    =   df_input['wind_speed'][day*timesteps + step]
                    df_training['WS+1'][day*timesteps + step]  =   df_input['wind_speed'][day*timesteps + step]
                    df_training['Pr-1'][day*timesteps + step]  =   df_input['tp6'][day*timesteps + step-1]
                    df_training['Pr'][day*timesteps + step]    =   df_input['tp6'][day*timesteps + step]
                    df_training['Pr+1'][day*timesteps + step]  =   df_input['tp6'][day*timesteps + step]

                    df_target['temp'][day*timesteps + step]                     =   df_true['temp'][day + step] 
                    df_target['wind_direction'][day*timesteps + step]           =   df_true['wind_direction'][day + step] 
                    df_target['wind_speed'][day*timesteps + step]               =   df_true['wind_speed'][day + step] 
                    df_target['precip_quantity_6hour'][day*timesteps + step]    =   df_true['precip_quantity_6hour'][day + step]

                    continue

                df_training['T-1'][day*timesteps + step]   =   df_input['t2m'][day*timesteps + step-1]
                df_training['T'][day*timesteps + step]     =   df_input['t2m'][day*timesteps + step]
                df_training['T+1'][day*timesteps + step]   =   df_input['t2m'][day*timesteps + step+1]
                df_training['WD-1'][day*timesteps + step]  =   df_input['wind_direction'][day*timesteps + step-1]
                df_training['WD'][day*timesteps + step]    =   df_input['wind_direction'][day*timesteps + step]
                df_training['WD+1'][day*timesteps + step]  =   df_input['wind_direction'][day*timesteps + step+1]
                df_training['WS-1'][day*timesteps + step]  =   df_input['wind_speed'][day*timesteps + step-1]
                df_training['WS'][day*timesteps + step]    =   df_input['wind_speed'][day*timesteps + step]
                df_training['WS+1'][day*timesteps + step]  =   df_input['wind_speed'][day*timesteps + step+1]
                df_training['Pr-1'][day*timesteps + step]  =   df_input['tp6'][day*timesteps + step-1]
                df_training['Pr'][day*timesteps + step]    =   df_input['tp6'][day*timesteps + step]
                df_training['Pr+1'][day*timesteps + step]  =   df_input['tp6'][day*timesteps + step+1]

                df_target['temp'][day*timesteps + step]                     =   df_true['temp'][day + step] 
                df_target['wind_direction'][day*timesteps + step]           =   df_true['wind_direction'][day + step] 
                df_target['wind_speed'][day*timesteps + step]               =   df_true['wind_speed'][day + step] 
                df_target['precip_quantity_6hour'][day*timesteps + step]    =   df_true['precip_quantity_6hour'][day + step] 


KeyError: 0

In [ ]:
#  DAS IST MUMPITZ






In [ ]:
df_training.to_csv('training.csv', index = False)
df_target.to_csv('target.csv', index = False)


For the first week, we are only supposed to calculate the quantiles of the given data. 
This we do for four variables, temperature, precipitation, windspeed and relative humidity.

In [ ]:
df = pd.read_csv('Forecast_08_04_2023.csv')


df['wind'] = np.sqrt(df['u10']**2 + df['v10']**2)
d2m = df['d2m']
t2m = df['t2m']
df['rh'] = 100* (np.exp(17.625*d2m / (243.04 + d2m))) / (np.exp(17.625*t2m / (243.04 + t2m)))
df.pop('u10')
df.pop('v10')
df.pop('latitude')
df.pop('longitude')
df.pop('d2m')
df.pop('number')
df['t2m'] = df['t2m']-273.15
df.head()


NameError: ignored

In [ ]:
df.tail()

,time,t2m,tp,wind,rh
1045,2023-04-06 00:00:00,276.32208,8.452170,2.579135,92.476333
1046,2023-04-06 00:00:00,274.98550,9.049661,2.007462,97.012251
1047,2023-04-06 00:00:00,274.97400,9.169874,2.019161,94.859643
1048,2023-04-06 00:00:00,273.91232,6.438590,1.852202,97.294717
1049,2023-04-06 00:00:00,273.63214,9.265759,2.125684,97.125641


In [ ]:
# Define the bin size
bin_size = 50
no_quantiles = 5
no_variables = 4

# Calculate the number of bins, we dont want number 0
n_bins = int(len(df) / bin_size - 1)

arr = np.zeros((80,no_quantiles))

for i in range(20):
    start = (i) * bin_size    # we dont want number 0
    end = start + bin_size
    arr[i,0] = df.iloc[start:end]['wind'].quantile(0.025)
    arr[i,1] = df.iloc[start:end]['wind'].quantile(0.25)
    arr[i,2] = df.iloc[start:end]['wind'].quantile(0.5)
    arr[i,3] = df.iloc[start:end]['wind'].quantile(0.75)
    arr[i,4] = df.iloc[start:end]['wind'].quantile(0.975)

    arr[i+20,0] = df.iloc[start:end]['t2m'].quantile(0.025)
    arr[i+20,1] = df.iloc[start:end]['t2m'].quantile(0.25)
    arr[i+20,2] = df.iloc[start:end]['t2m'].quantile(0.5)
    arr[i+20,3] = df.iloc[start:end]['t2m'].quantile(0.75)
    arr[i+20,4] = df.iloc[start:end]['t2m'].quantile(0.975)

    arr[i+40,0] = df.iloc[start:end]['rh'].quantile(0.025)
    arr[i+40,1] = df.iloc[start:end]['rh'].quantile(0.25)
    arr[i+40,2] = df.iloc[start:end]['rh'].quantile(0.5)
    arr[i+40,3] = df.iloc[start:end]['rh'].quantile(0.75)
    arr[i+40,4] = df.iloc[start:end]['rh'].quantile(0.975)

    arr[i+60,0] = df.iloc[start:end]['tp'].quantile(0.025)
    arr[i+60,1] = df.iloc[start:end]['tp'].quantile(0.25)
    arr[i+60,2] = df.iloc[start:end]['tp'].quantile(0.5)
    arr[i+60,3] = df.iloc[start:end]['tp'].quantile(0.75)
    arr[i+60,4] = df.iloc[start:end]['tp'].quantile(0.975)


print(arr)


[[1.11426041e+00 1.53825466e+00 1.78466171e+00 1.91574718e+00
  2.12156037e+00]
 [8.25536862e-01 1.18286028e+00 1.41511665e+00 1.66415366e+00
  2.10447769e+00]
 [5.87629416e-01 1.16389570e+00 1.38283915e+00 1.51558879e+00
  1.84009587e+00]
 [1.07640889e+00 1.52484276e+00 1.79751446e+00 2.12498709e+00
  2.43338264e+00]
 [1.36905027e+00 1.77455605e+00 1.88096050e+00 2.11895718e+00
  2.44928765e+00]
 [4.83093617e-01 1.56221835e+00 1.83200310e+00 2.14973704e+00
  2.75980501e+00]
 [1.20724433e+00 1.45295309e+00 1.61387782e+00 1.82977486e+00
  2.21276264e+00]
 [2.40197534e+00 2.89217573e+00 3.26464333e+00 3.71184735e+00
  4.19988582e+00]
 [3.63533005e+00 4.06467389e+00 4.26260414e+00 4.53491723e+00
  4.98130956e+00]
 [4.84209927e+00 5.64659039e+00 5.97155513e+00 6.27638254e+00
  6.89664944e+00]
 [4.73111690e+00 5.44457385e+00 5.75110903e+00 6.58976312e+00
  7.33296039e+00]
 [3.01178416e+00 5.12225227e+00 5.92638640e+00 6.80073644e+00
  8.14229516e+00]
 [3.26021290e+00 4.56744504e+00 5.209647

In [ ]:
data = {
    'date': ['2023-04-08'] * 80,
    'target': ['wind'] * 20 + ['t2m'] * 20 + ['rh'] * 20 + ['precip'] * 20,
    'horizon': [0] * 80,
    'q0.025': arr[:, 0],
    'q0.25':  arr[:, 1],
    'q0.5':  arr[:, 2],
    'q0.75':  arr[:, 3],
    'q0.975':  arr[:, 4]
}

df_new = pd.DataFrame(data)
for i in range(20):
    df_new['horizon'][i] = 6+i*6 
    df_new['horizon'][i+20] = 6+i*6
    df_new['horizon'][i+40] = 6+i*6
    df_new['horizon'][i+60] = 6+i*6

df_new.to_csv('20230408_PaoloConte.csv', index = False)



<ipython-input-20-946d0598ff64>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['horizon'][i] = 6+i*6
<ipython-input-20-946d0598ff64>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['horizon'][i+20] = 6+i*6
<ipython-input-20-946d0598ff64>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['horizon'][i+40] = 6+i*6
<ipython-input-20-946d0598ff64>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy o